# ArcadeDB Python Bindings - Complete Guide

This notebook provides a comprehensive walkthrough of ArcadeDB's Python bindings, covering all major features:

- 📁 Database creation and management
- 📝 Document operations (CRUD)
- 🔄 Graph operations (vertices & edges)
- 💾 Transactions
- 🔍 Multiple query languages (SQL, Cypher, Gremlin)
- 🎯 Vector storage for embeddings
- 🌐 Server mode with Studio UI
- 🧹 Resource management best practices

**Requirements**: 
- ArcadeDB Python bindings installed (`pip install arcadedb-embedded-full`)
- Java Runtime Environment (JRE) installed

## 1. Setup & Imports

First, let's import the library and check the version:

**Note**: If you encounter Java version issues, ensure `JAVA_HOME` is set correctly:

In [1]:
import arcadedb_embedded as arcadedb
import tempfile
import shutil
from pathlib import Path

# Check version
print(f"ArcadeDB Python Bindings Version: {arcadedb.__version__}")
print(f"Module Location: {arcadedb.__file__}")

# Create a temporary directory for our examples
TEMP_DIR = Path(tempfile.mkdtemp(prefix="arcadedb_demo_"))
print(f"\n📁 Using temporary directory: {TEMP_DIR}")

ArcadeDB Python Bindings Version: 25.10.1
Module Location: /home/tk/repos/arcadedb/.venv/lib/python3.11/site-packages/arcadedb_embedded/__init__.py

📁 Using temporary directory: /tmp/arcadedb_demo_b4t5h7v9


## 2. Creating a Database

ArcadeDB supports embedded mode (direct file access) with no network required. Let's create a database:

In [2]:
# Create database using context manager (recommended - automatic cleanup)
db_path = TEMP_DIR / "mydb"
db = arcadedb.create_database(str(db_path))

print(f"✅ Database created at: {db_path}")
print(f"📊 Database is open: {db.is_open()}")
print(f"🏷️  Database name: {db.get_name()}")

✅ Database created at: /tmp/arcadedb_demo_b4t5h7v9/mydb
📊 Database is open: True
🏷️  Database name: mydb


[To redirect Truffle log output to a file use one of the following options:
* '--log.file=<path>' if the option is passed using a guest language launcher.
* '-Dpolyglot.log.file=<path>' if the option is passed using the host Java launcher.
* Configure logging using the polyglot embedding API.]
[engine] WARNING: The polyglot engine uses a fallback runtime that does not support runtime compilation to native code.
Execution without runtime compilation will negatively impact the guest application performance.
The following cause was found: JVMCI is not enabled for this JVM. Enable JVMCI using -XX:+EnableJVMCI.
For more information see: https://www.graalvm.org/latest/reference-manual/embed-languages/#runtime-optimization-support.
To disable this warning use the '--engine.WarnInterpreterOnly=false' option or the '-Dpolyglot.engine.WarnInterpreterOnly=false' system property.

2025-10-18 21:48:02.438 WARNI [PaginatedComponentFile] Unable to disable channel close on interrupt: Unable to make fi

## 3. Document Operations

Let's create a schema and perform CRUD operations on documents:

In [3]:
# Create document type (schema)
with db.transaction():
    db.command("sql", "CREATE DOCUMENT TYPE Person")
    db.command("sql", "CREATE PROPERTY Person.name STRING")
    db.command("sql", "CREATE PROPERTY Person.age INTEGER")
    db.command("sql", "CREATE PROPERTY Person.email STRING")

print("✅ Schema created: Person type with name, age, email properties")

✅ Schema created: Person type with name, age, email properties


In [4]:
# Insert documents (CREATE)
with db.transaction():
    db.command("sql", "INSERT INTO Person SET name = 'Alice Johnson', age = 30, email = 'alice@example.com'")
    db.command("sql", "INSERT INTO Person SET name = 'Bob Smith', age = 25, email = 'bob@example.com'")
    db.command("sql", "INSERT INTO Person SET name = 'Charlie Brown', age = 35, email = 'charlie@example.com'")
    db.command("sql", "INSERT INTO Person SET name = 'Diana Prince', age = 28, email = 'diana@example.com'")

print("✅ Inserted 4 people into the database")

✅ Inserted 4 people into the database


In [5]:
# Query documents (READ)
result = db.query("sql", "SELECT FROM Person WHERE age > 25")

print(f"📊 Found {len(list(result))} people over 25 years old:\n")

# Re-query to iterate (ResultSet can only be iterated once)
result = db.query("sql", "SELECT FROM Person WHERE age > 25")
for person in result:
    name = person.get_property('name')
    age = person.get_property('age')
    email = person.get_property('email')
    print(f"  • {name} (age {age}) - {email}")

📊 Found 3 people over 25 years old:

  • Alice Johnson (age 30) - alice@example.com
  • Charlie Brown (age 35) - charlie@example.com
  • Diana Prince (age 28) - diana@example.com


In [6]:
# Update documents (UPDATE)
with db.transaction():
    db.command("sql", "UPDATE Person SET age = 31 WHERE name = 'Alice Johnson'")

print("✅ Updated Alice's age to 31")

# Verify update
result = db.query("sql", "SELECT FROM Person WHERE name = 'Alice Johnson'")
alice = list(result)[0]
print(f"   Alice's new age: {alice.get_property('age')}")

✅ Updated Alice's age to 31
   Alice's new age: 31


## 4. Working with ResultSet

The ResultSet object provides various methods to work with query results:

In [7]:
result = db.query("sql", "SELECT FROM Person WHERE name = 'Bob Smith'")
bob = list(result)[0]

# Check properties
print("🔍 Property Methods:")
print(f"  has_property('name'): {bob.has_property('name')}")
print(f"  has_property('salary'): {bob.has_property('salary')}")

# Get all property names
print(f"\n📋 Property names: {bob.get_property_names()}")

# Convert to dictionary
print(f"\n📦 to_dict(): {bob.to_dict()}")

# Convert to JSON string
print(f"\n📄 to_json():\n{bob.to_json()}")

🔍 Property Methods:
  has_property('name'): True
  has_property('salary'): False

📋 Property names: ['name', 'age', 'email']

📦 to_dict(): {'name': 'Bob Smith', 'age': 25, 'email': 'bob@example.com'}

📄 to_json():
{"name":"Bob Smith","age":25,"email":"bob@example.com","@props":"name:7,age:1,email:7","@cat":"d","@type":"Person","@rid":"#1:1"}


## 5. Graph Operations

ArcadeDB is a multi-model database. Let's create a graph with vertices and edges:

In [8]:
# Create graph schema
with db.transaction():
    db.command("sql", "CREATE VERTEX TYPE User")
    db.command("sql", "CREATE EDGE TYPE Follows")
    db.command("sql", "CREATE EDGE TYPE Likes")

print("✅ Created graph schema: User vertices with Follows and Likes edges")

✅ Created graph schema: User vertices with Follows and Likes edges


In [9]:
# Create vertices
with db.transaction():
    db.command("sql", "CREATE VERTEX User SET username = 'alice', followers = 0")
    db.command("sql", "CREATE VERTEX User SET username = 'bob', followers = 0")
    db.command("sql", "CREATE VERTEX User SET username = 'charlie', followers = 0")
    db.command("sql", "CREATE VERTEX User SET username = 'diana', followers = 0")

print("✅ Created 4 user vertices")

✅ Created 4 user vertices


In [10]:
# Create edges (relationships)
with db.transaction():
    # Alice follows Bob and Charlie
    db.command("sql", """
        CREATE EDGE Follows
        FROM (SELECT FROM User WHERE username = 'alice')
        TO (SELECT FROM User WHERE username = 'bob')
    """)
    db.command("sql", """
        CREATE EDGE Follows
        FROM (SELECT FROM User WHERE username = 'alice')
        TO (SELECT FROM User WHERE username = 'charlie')
    """)
    
    # Bob follows Charlie
    db.command("sql", """
        CREATE EDGE Follows
        FROM (SELECT FROM User WHERE username = 'bob')
        TO (SELECT FROM User WHERE username = 'charlie')
    """)
    
    # Diana follows Alice
    db.command("sql", """
        CREATE EDGE Follows
        FROM (SELECT FROM User WHERE username = 'diana')
        TO (SELECT FROM User WHERE username = 'alice')
    """)

print("✅ Created follow relationships")

✅ Created follow relationships


In [11]:
# Graph traversal - Who does Alice follow?
result = db.query("sql", """
    SELECT expand(out('Follows').username)
    FROM User
    WHERE username = 'alice'
""")

print("👤 Alice follows:")
for record in result:
    username = record.get_property('value')
    print(f"  • {username}")

👤 Alice follows:
  • charlie
  • bob


In [12]:
# Who follows Alice? (incoming edges)
result = db.query("sql", """
    SELECT expand(in('Follows').username)
    FROM User
    WHERE username = 'alice'
""")

print("👥 Alice's followers:")
for record in result:
    username = record.get_property('value')
    print(f"  • {username}")

👥 Alice's followers:
  • diana


## 6. Transactions

Transactions ensure ACID compliance. Let's see how to use them:

In [13]:
# Context manager (recommended) - automatic commit/rollback
print("💾 Transaction with context manager:")

try:
    with db.transaction():
        db.command("sql", "INSERT INTO Person SET name = 'Eve Wilson', age = 40, email = 'eve@example.com'")
        db.command("sql", "INSERT INTO Person SET name = 'Frank Miller', age = 45, email = 'frank@example.com'")
        # Transaction auto-commits if no exceptions
    print("  ✅ Transaction committed successfully")
except Exception as e:
    print(f"  ❌ Transaction failed: {e}")

# Verify
result = db.query("sql", "SELECT count(*) as count FROM Person")
count = list(result)[0].get_property('count')
print(f"  📊 Total people in database: {count}")

💾 Transaction with context manager:
  ✅ Transaction committed successfully
  📊 Total people in database: 6


In [14]:
# Transaction with intentional rollback
print("\n🔄 Transaction with rollback:")

try:
    with db.transaction():
        db.command("sql", "INSERT INTO Person SET name = 'Grace Hopper', age = 50, email = 'grace@example.com'")
        print("  Inserted Grace Hopper")
        
        # Simulate an error
        raise Exception("Intentional error to trigger rollback")
        
except Exception as e:
    print(f"  ⚠️  Exception caught: {e}")
    print("  ↩️  Transaction automatically rolled back")

# Verify Grace was NOT inserted
result = db.query("sql", "SELECT FROM Person WHERE name = 'Grace Hopper'")
found = len(list(result))
print(f"  📊 Grace Hopper records found: {found} (should be 0)")


🔄 Transaction with rollback:
  Inserted Grace Hopper
  ⚠️  Exception caught: Intentional error to trigger rollback
  ↩️  Transaction automatically rolled back
  📊 Grace Hopper records found: 0 (should be 0)


## 7. Multiple Query Languages

ArcadeDB supports multiple query languages. Let's try Cypher and Gremlin (if available):

In [15]:
# Cypher queries (if Cypher engine is available)
print("🔷 Cypher Query Language:\n")

try:
    # Query using Cypher
    result = db.query("cypher", "MATCH (u:User) WHERE u.username = 'alice' RETURN u.username as username")
    users = list(result)
    
    if users:
        print(f"  ✅ Found user via Cypher: {users[0].get_property('username')}")
    
except Exception as e:
    if "Query engine 'cypher' was not found" in str(e):
        print("  ⏭️  Cypher not available in this distribution (need full distribution)")
    else:
        print(f"  ❌ Error: {e}")

🔷 Cypher Query Language:

  ⏭️  Cypher not available in this distribution (need full distribution)


In [16]:
# Gremlin queries (only in full distribution)
print("\n🟢 Gremlin Query Language:\n")

try:
    # Query using Gremlin
    result = db.query("gremlin", "g.V().hasLabel('User').has('username', 'bob').values('username')")
    usernames = list(result)
    
    if usernames:
        print(f"  ✅ Found user via Gremlin: {usernames[0].get_property('value')}")
    
except Exception as e:
    if "gremlin" in str(e).lower():
        print("  ⏭️  Gremlin not available (requires full distribution)")
    else:
        print(f"  ❌ Error: {e}")


🟢 Gremlin Query Language:

  ⏭️  Gremlin not available (requires full distribution)


## 8. Vector Storage with HNSW Similarity Search

Store vector embeddings with HNSW (Hierarchical Navigable Small World) indexing for efficient nearest-neighbor similarity search.

**Installation**: For optimal performance with vectors, install with NumPy support:
```bash
pip install arcadedb-embedded-full[vector]  # Includes NumPy
```

The vector API works with both NumPy arrays and plain Python lists!

In [17]:
# Try to import NumPy (optional, but recommended for performance)
try:
    import numpy as np
    use_numpy = True
    print("✅ NumPy available - will use np.array for vectors")
except ImportError:
    use_numpy = False
    print("⚠️  NumPy not available - will use Python lists (install with: pip install numpy)")

# Create vertex type for vector embeddings
with db.transaction():
    db.command("sql", "CREATE VERTEX TYPE EmbeddingNode")
    db.command("sql", "CREATE PROPERTY EmbeddingNode.name STRING")
    # Use ARRAY_OF_FLOATS for vector property (required for HNSW)
    db.command("sql", "CREATE PROPERTY EmbeddingNode.vector ARRAY_OF_FLOATS")
    # Create unique index on name for lookups
    db.command("sql", "CREATE INDEX ON EmbeddingNode (name) UNIQUE")

print("✅ Created EmbeddingNode vertex type for storing vectors with HNSW indexing")

✅ NumPy available - will use np.array for vectors
✅ Created EmbeddingNode vertex type for storing vectors with HNSW indexing
✅ Created EmbeddingNode vertex type for storing vectors with HNSW indexing


In [18]:
# Insert sample word embeddings (4-dimensional for simplicity)
# In production, embeddings would typically be 300-1536 dimensions

if use_numpy:
    embeddings = [
        ("king", np.array([0.5, 0.3, 0.1, 0.2])),
        ("queen", np.array([0.52, 0.32, 0.08, 0.18])),  # Similar to king
        ("man", np.array([0.48, 0.25, 0.15, 0.22])),
        ("woman", np.array([0.50, 0.28, 0.12, 0.20])),
        ("cat", np.array([0.1, 0.8, 0.6, 0.3])),        # Different cluster
        ("dog", np.array([0.12, 0.82, 0.58, 0.32])),    # Similar to cat
    ]
else:
    embeddings = [
        ("king", [0.5, 0.3, 0.1, 0.2]),
        ("queen", [0.52, 0.32, 0.08, 0.18]),
        ("man", [0.48, 0.25, 0.15, 0.22]),
        ("woman", [0.50, 0.28, 0.12, 0.20]),
        ("cat", [0.1, 0.8, 0.6, 0.3]),
        ("dog", [0.12, 0.82, 0.58, 0.32]),
    ]

with db.transaction():
    for name, vector in embeddings:
        # Use the helper function to convert to Java float array
        # Works with both NumPy arrays and Python lists!
        java_vector = arcadedb.to_java_float_array(vector)
        
        # Create vertex with vector property
        java_db = db._java_db
        vertex = java_db.newVertex("EmbeddingNode")
        vertex.set("name", name)
        vertex.set("vector", java_vector)
        vertex.save()

print(f"✅ Inserted {len(embeddings)} word embeddings")
print(f"   Format: {'NumPy arrays' if use_numpy else 'Python lists'}")

✅ Inserted 6 word embeddings
   Format: NumPy arrays


In [19]:
# Create HNSW vector index using the simplified Python API
print("Creating HNSW vector index with simplified API...\n")

# Create the index - Python API handles all the Java complexity!
with db.transaction():
    index = db.create_vector_index(
        vertex_type="EmbeddingNode",
        vector_property="vector",
        dimensions=4,
        id_property="name",
        distance_function="cosine",  # or "euclidean", "inner_product"
        m=16,                        # HNSW M parameter
        ef=128,                      # Search quality parameter
        max_items=1000
    )

print("  ✓ Created HNSW vector index")

# Index existing vertices
print("  Indexing existing vertices...")
result = db.query("sql", "SELECT FROM EmbeddingNode")
indexed_count = 0

with db.transaction():
    for record in result:
        vertex = record._java_result.getElement().get().asVertex()
        index.add_vertex(vertex)
        indexed_count += 1

print(f"  ✓ Indexed {indexed_count} vertices")
print("\n🎯 HNSW vector index ready for similarity search!")

Creating HNSW vector index with simplified API...

  ✓ Created HNSW vector index
  Indexing existing vertices...
  ✓ Indexed 6 vertices

🎯 HNSW vector index ready for similarity search!
  ✓ Created HNSW vector index
  Indexing existing vertices...
  ✓ Indexed 6 vertices

🎯 HNSW vector index ready for similarity search!


In [20]:
# Perform nearest-neighbor similarity search using the simplified API
print("Testing nearest-neighbor similarity search...\n")

# Search for neighbors of "king" - should find "queen", "man", "woman"
if use_numpy:
    king_vector = np.array([0.5, 0.3, 0.1, 0.2])
else:
    king_vector = [0.5, 0.3, 0.1, 0.2]

# find_nearest() accepts both NumPy arrays and Python lists!
neighbors = index.find_nearest(king_vector, k=3)

# Extract names from results
neighbor_names = [str(vertex.get("name")) for vertex, distance in neighbors]

print(f"  3 nearest neighbors to 'king': {neighbor_names}")
print(f"  ✓ Expected: queen, man, woman (similar embeddings)")

# Show distances
print("\n  Detailed results:")
for vertex, distance in neighbors:
    name = str(vertex.get("name"))
    print(f"    • {name}: distance = {distance:.4f}")

# Search for neighbors of "cat" - should find "dog"
if use_numpy:
    cat_vector = np.array([0.1, 0.8, 0.6, 0.3])
else:
    cat_vector = [0.1, 0.8, 0.6, 0.3]

neighbors = index.find_nearest(cat_vector, k=2)
neighbor_names = [str(vertex.get("name")) for vertex, distance in neighbors]

print(f"\n  2 nearest neighbors to 'cat': {neighbor_names}")
print(f"  ✓ Expected: dog (different cluster from king/queen)")

print("\n✅ HNSW vector similarity search working correctly!")
print("💡 This enables semantic search, recommendation systems, and RAG applications")
print(f"💡 API works seamlessly with {'NumPy arrays' if use_numpy else 'Python lists'}!")

Testing nearest-neighbor similarity search...

  3 nearest neighbors to 'king': ['king', 'woman', 'queen']
  ✓ Expected: queen, man, woman (similar embeddings)

  Detailed results:
    • king: distance = 0.0000
    • woman: distance = 0.0010
    • queen: distance = 0.0016

  2 nearest neighbors to 'cat': ['cat', 'dog']
  ✓ Expected: dog (different cluster from king/queen)

✅ HNSW vector similarity search working correctly!
💡 This enables semantic search, recommendation systems, and RAG applications
💡 API works seamlessly with NumPy arrays!

  2 nearest neighbors to 'cat': ['cat', 'dog']
  ✓ Expected: dog (different cluster from king/queen)

✅ HNSW vector similarity search working correctly!
💡 This enables semantic search, recommendation systems, and RAG applications
💡 API works seamlessly with NumPy arrays!


In [21]:
# Demonstrate utility functions for vector conversions
print("\n🔧 Vector Utility Functions:\n")

# Query a vector and convert it back to Python
result = db.query("sql", "SELECT FROM EmbeddingNode WHERE name = 'king'")
king_node = list(result)[0]
java_vector = king_node.get_property('vector')

# Convert to Python array (with optional NumPy)
python_vector = arcadedb.to_python_array(java_vector, use_numpy=use_numpy)

print(f"  Original vector type: {type(python_vector)}")
print(f"  Vector values: {python_vector}")

if use_numpy:
    print(f"  NumPy shape: {python_vector.shape}")
    print(f"  NumPy dtype: {python_vector.dtype}")

# Convert back to Java array
java_again = arcadedb.to_java_float_array(python_vector)
print(f"\n  ✓ Round-trip conversion: Python → Java → Python → Java")
print("  💡 These utilities make it easy to work with vectors in your preferred format!")


🔧 Vector Utility Functions:

  Original vector type: <class 'numpy.ndarray'>
  Vector values: [0.5 0.3 0.1 0.2]
  NumPy shape: (4,)
  NumPy dtype: float32

  ✓ Round-trip conversion: Python → Java → Python → Java
  💡 These utilities make it easy to work with vectors in your preferred format!


## 9. Server Mode with Studio UI

For development and visualization, you can start an HTTP server with the Studio web interface:

In [22]:
# Note: Server mode requires minimal or full distribution
print("🌐 Server Mode Example:\n")
print("Server mode is not demonstrated in this notebook to avoid port conflicts.")
print("Here's how you would use it in a script:\n")

example_code = '''
from arcadedb_embedded import create_server

# Start server with context manager
with create_server("./databases", root_password="mypassword") as server:
    print(f"Studio UI: {server.get_studio_url()}")  # http://localhost:2480
    print(f"HTTP Port: {server.get_http_port()}")
    
    # Create or get database through server
    db = server.create_database("mydb")
    
    # Use database normally
    with db.transaction():
        db.command("sql", "CREATE DOCUMENT TYPE Product")
    
    # Database is accessible via HTTP API while server runs
    # Server automatically stops when exiting context
'''

print(example_code)

🌐 Server Mode Example:

Server mode is not demonstrated in this notebook to avoid port conflicts.
Here's how you would use it in a script:


from arcadedb_embedded import create_server

# Start server with context manager
with create_server("./databases", root_password="mypassword") as server:
    print(f"Studio UI: {server.get_studio_url()}")  # http://localhost:2480
    print(f"HTTP Port: {server.get_http_port()}")

    # Create or get database through server
    db = server.create_database("mydb")

    # Use database normally
    with db.transaction():
        db.command("sql", "CREATE DOCUMENT TYPE Product")

    # Database is accessible via HTTP API while server runs
    # Server automatically stops when exiting context



## 10. Resource Management & Cleanup

ArcadeDB provides multiple ways to ensure proper resource cleanup:

In [23]:
print("🧹 Resource Management Best Practices:\n")

print("1️⃣  Context Manager (Recommended):")
print("   with arcadedb.create_database(path) as db:")
print("       # Use database")
print("   # Automatically closed\n")

print("2️⃣  Manual Close:")
print("   db = arcadedb.create_database(path)")
print("   try:")
print("       # Use database")
print("   finally:")
print("       db.close()\n")

print("3️⃣  Automatic Cleanup (Finalizer):")
print("   db = arcadedb.create_database(path)")
print("   # Use database")
print("   # Python garbage collector will call db.__del__() automatically\n")

print("✅ All methods ensure:")
print("   • Database files are closed properly")
print("   • No memory leaks")
print("   • No zombie processes (JVM runs as threads in Python process)")

🧹 Resource Management Best Practices:

1️⃣  Context Manager (Recommended):
   with arcadedb.create_database(path) as db:
       # Use database
   # Automatically closed

2️⃣  Manual Close:
   db = arcadedb.create_database(path)
   try:
       # Use database
   finally:
       db.close()

3️⃣  Automatic Cleanup (Finalizer):
   db = arcadedb.create_database(path)
   # Use database
   # Python garbage collector will call db.__del__() automatically

✅ All methods ensure:
   • Database files are closed properly
   • No memory leaks
   • No zombie processes (JVM runs as threads in Python process)


In [24]:
# Close our database properly
db.close()
print("✅ Database closed")
print(f"📊 Database is still open: {db.is_open()}")

✅ Database closed
📊 Database is still open: False


## 11. Cleanup Temporary Files

Let's clean up the temporary directory we created:

In [25]:
# Clean up the temporary directory
shutil.rmtree(TEMP_DIR, ignore_errors=True)
print(f"🗑️  Cleaned up temporary directory: {TEMP_DIR}")

🗑️  Cleaned up temporary directory: /tmp/arcadedb_demo_b4t5h7v9


## 📚 Summary

This notebook demonstrated all major features of ArcadeDB Python bindings:

✅ **Database Operations**
- Creating and managing databases
- Document CRUD operations
- Schema management

✅ **Graph Operations**
- Creating vertices and edges
- Graph traversals (in/out relationships)
- Multi-model data

✅ **Transactions**
- Context manager (automatic commit/rollback)
- ACID compliance
- Error handling

✅ **Query Languages**
- SQL (all distributions)
- Cypher (full distribution)
- Gremlin (full distribution)

✅ **Vector Storage with HNSW (NEW! 🎉)**
- Simplified Python API with NumPy support
- `to_java_float_array()` and `to_python_array()` utilities
- `VectorIndex` class for similarity search
- `db.create_vector_index()` for easy index creation
- Works with both NumPy arrays and Python lists
- Cosine, Euclidean, and Inner Product distance functions
- Perfect for embeddings, semantic search, RAG, and ML applications

✅ **Resource Management**
- Context managers
- Manual cleanup
- Automatic finalizers

---

### Next Steps

1. **Install the appropriate distribution**:
   - `arcadedb-embedded-headless` - Core features only (94 MB)
   - `arcadedb-embedded-minimal` - + Server & Studio UI (97 MB)
   - `arcadedb-embedded-full` - + Cypher & Gremlin (158 MB)
   
2. **For vector operations, install with NumPy**:
   ```bash
   pip install arcadedb-embedded-full[vector]  # Includes NumPy
   ```

3. **Explore the documentation**:
   - [ArcadeDB Docs](https://docs.arcadedb.com)
   - [Python Bindings README](../README.md)
   - [HNSW Vector Index](https://docs.arcadedb.com/#Vector-Embedding)

4. **Build your application**:
   - Use embedded mode for direct file access
   - Use server mode for web UI and HTTP API
   - Combine multiple data models as needed
   - Leverage HNSW with the simplified Python API for semantic search and ML applications

**Happy coding! 🚀**